# OS grid reference conversion
Convert the 10km or 1km OS grid squares to a lat/lon pair

In [ ]:
import numpy as np
import pandas as pd
from math import sqrt, pi, sin, cos, tan, atan2 as arctan2
import re

This converts grid references with two leading letters to fully-numeric OSGB36 references.

In [ ]:
# From http://all-geo.org/volcan01010/2012/11/change-coordinates-with-pyproj/

# Region codes for 100 km grid squares.
regions=[['HL','HM','HN','HO','HP','JL','JM'],
      ['HQ','HR','HS','HT','HU','JQ','JR'],
      ['HV','HW','HX','HY','HZ','JV','JW'],
      ['NA','NB','NC','ND','NE','OA','OB'],
      ['NF','NG','NH','NJ','NK','OF','OG'],
      ['NL','NM','NN','NO','NP','OL','OM'],
      ['NQ','NR','NS','NT','NU','OQ','OR'],
      ['NV','NW','NX','NY','NZ','OV','OW'],
      ['SA','SB','SC','SD','SE','TA','TB'],
      ['SF','SG','SH','SJ','SK','TF','TG'],
      ['SL','SM','SN','SO','SP','TL','TM'],
      ['SQ','SR','SS','ST','SU','TQ','TR'],
      ['SV','SW','SX','SY','SZ','TV','TW']]
# Reshuffle so indices correspond to offsets
regions=np.array( [ regions[x] for x in range(12,-1,-1) ] )
regions=regions.transpose()
 
#-------------------------------------------------------------------------------
def to_osgb36(coords):
    """Reformat British National Grid references to OSGB36 numeric coordinates.
    Grid references can be 4, 6, 8 or 10 figures.  Returns a tuple of x, y.
 
    Examples:
 
    Single value
    >>> to_osgb36('NT2755072950')
    (327550, 672950)
 
    For multiple values, use the zip function
    >>> gridrefs = ['HU431392', 'SJ637560', 'TV374354']
    >>> xy=to_osgb36(gridrefs)
    >>> x, y = zip(*xy)
    >>> x
    (443100, 363700, 537400)
    >>> y
    (1139200, 356000, 35400)
    """
    #
    # Check for individual coord, or list, tuple or array of coords
    #
    if type(coords)==list:
        return [to_osgb36(c) for c in coords]
    elif type(coords)==tuple:
        return tuple([to_osgb36(c) for c in coords])
    elif type(coords)==type(np.array('string')):
        return np.array([ to_osgb36(str(c))  for c in list(coords) ])
    #
    # Input is grid reference...
    #
    elif type(coords)==str and re.match(r'^[A-Za-z]{2}(\d{2}|\d{6}|\d{8}|\d{10})$', coords):
        region=coords[0:2].upper()
        x_box, y_box = np.where(regions==region)
        try: # Catch bad region codes
            x_offset = 100000 * x_box[0] # Convert index in 'regions' to offset
            y_offset = 100000 * y_box[0]
        except IndexError:
            raise ValueError('Invalid 100km grid square code')
        nDigits = (len(coords)-2)//2
        factor = 10**(5-nDigits)
        x,y = (int(coords[2:2+nDigits])*factor + x_offset,
               int(coords[2+nDigits:2+2*nDigits])*factor + y_offset)
        return x, y
    #
    # Catch invalid input
    #
    else:
        raise TypeError('Valid inputs are 6,8 or 10-fig references as strings e.g.'
                        '"NN123321", or lists/tuples/arrays of strings.')
 
#-------------------------------------------------------------------------------
def from_osgb36(coords, nDigits=6):
    """Reformat OSGB36 numeric coordinates to British National Grid references.
    Grid references can be 4, 6, 8 or 10 fig, specified by the nDigits keyword.
 
    Examples:
 
    Single value
    >>> from_osgb36((327550, 672950))
    'NT276730'
 
    For multiple values, use the zip function
    >>> x = [443143, 363723, 537395]
    >>> y = [1139158, 356004, 35394]
    >>> xy = zip(x, y)
    >>> from_osgb36(xy, nDigits=4)
    ['HU4339', 'SJ6456', 'TV3735']
    """
    if (type(coords)==list):
        return [from_osgb36(c, nDigits=nDigits) for c in coords]
    #
    # Input is a tuple of numeric coordinates...
    #
    elif type(coords)==tuple:
        x, y = coords
        x_box=np.floor(x/100000.0)  # Convert offset to index in 'regions'
        y_box=np.floor(y/100000.0)
        x_offset=100000*x_box
        y_offset=100000*y_box
        try: # Catch coordinates outside the region
            region=regions[x_box, y_box]
        except IndexError:
            raise ValueError('Coordinate location outside UK region')
    #
    # Format the output based on nDigits
    #
        formats={4:'%s%02i%02i', 6:'%s%03i%03i', 8:'%s%04i%04i', 10:'%s%05i%05i'}
        factors={4:1000.0, 6:100.0, 8:10.0, 10:1.0}
        try: # Catch bad number of figures
            coords=formats[nDigits] % (region, np.floor((x-x_offset)/factors[nDigits]), 
                                       np.floor((y-y_offset)/factors[nDigits]))
        except KeyError:
            raise ValueError('Valid inputs for nDigits are 4, 6, 8 or 10')
        return coords
    #
    # Catch invalid input
    #
    else:
        raise TypeError('Valid inputs are x, y tuple e.g. (651409, 313177)')

This converts OSGB36 grid references to lat, lon pairs.

In [ ]:
# From http://hannahfry.co.uk/2012/02/01/converting-british-national-grid-to-latitude-and-longitude-ii/

def OSGB36toWGS84(E,N):

    #E, N are the British national grid coordinates - eastings and northings
    a, b = 6377563.396, 6356256.909     #The Airy 180 semi-major and semi-minor axes used for OSGB36 (m)
    F0 = 0.9996012717                   #scale factor on the central meridian
    lat0 = 49*pi/180                    #Latitude of true origin (radians)
    lon0 = -2*pi/180                    #Longtitude of true origin and central meridian (radians)
    N0, E0 = -100000, 400000            #Northing & easting of true origin (m)
    e2 = 1 - (b*b)/(a*a)                #eccentricity squared
    n = (a-b)/(a+b)

    #Initialise the iterative variables
    lat,M = lat0, 0

    while N-N0-M >= 0.00001: #Accurate to 0.01mm
        lat = (N-N0-M)/(a*F0) + lat;
        M1 = (1 + n + (5./4)*n**2 + (5./4)*n**3) * (lat-lat0)
        M2 = (3*n + 3*n**2 + (21./8)*n**3) * sin(lat-lat0) * cos(lat+lat0)
        M3 = ((15./8)*n**2 + (15./8)*n**3) * sin(2*(lat-lat0)) * cos(2*(lat+lat0))
        M4 = (35./24)*n**3 * sin(3*(lat-lat0)) * cos(3*(lat+lat0))
        #meridional arc
        M = b * F0 * (M1 - M2 + M3 - M4)          

    #transverse radius of curvature
    nu = a*F0/sqrt(1-e2*sin(lat)**2)

    #meridional radius of curvature
    rho = a*F0*(1-e2)*(1-e2*sin(lat)**2)**(-1.5)
    eta2 = nu/rho-1

    secLat = 1./cos(lat)
    VII = tan(lat)/(2*rho*nu)
    VIII = tan(lat)/(24*rho*nu**3)*(5+3*tan(lat)**2+eta2-9*tan(lat)**2*eta2)
    IX = tan(lat)/(720*rho*nu**5)*(61+90*tan(lat)**2+45*tan(lat)**4)
    X = secLat/nu
    XI = secLat/(6*nu**3)*(nu/rho+2*tan(lat)**2)
    XII = secLat/(120*nu**5)*(5+28*tan(lat)**2+24*tan(lat)**4)
    XIIA = secLat/(5040*nu**7)*(61+662*tan(lat)**2+1320*tan(lat)**4+720*tan(lat)**6)
    dE = E-E0

    #These are on the wrong ellipsoid currently: Airy1830. (Denoted by _1)
    lat_1 = lat - VII*dE**2 + VIII*dE**4 - IX*dE**6
    lon_1 = lon0 + X*dE - XI*dE**3 + XII*dE**5 - XIIA*dE**7

    #Want to convert to the GRS80 ellipsoid. 
    #First convert to cartesian from spherical polar coordinates
    H = 0 #Third spherical coord. 
    x_1 = (nu/F0 + H)*cos(lat_1)*cos(lon_1)
    y_1 = (nu/F0+ H)*cos(lat_1)*sin(lon_1)
    z_1 = ((1-e2)*nu/F0 +H)*sin(lat_1)

    #Perform Helmut transform (to go between Airy 1830 (_1) and GRS80 (_2))
    s = -20.4894*10**-6 #The scale factor -1
    tx, ty, tz = 446.448, -125.157, + 542.060 #The translations along x,y,z axes respectively
    rxs,rys,rzs = 0.1502,  0.2470,  0.8421  #The rotations along x,y,z respectively, in seconds
    rx, ry, rz = rxs*pi/(180*3600.), rys*pi/(180*3600.), rzs*pi/(180*3600.) #In radians
    x_2 = tx + (1+s)*x_1 + (-rz)*y_1 + (ry)*z_1
    y_2 = ty + (rz)*x_1  + (1+s)*y_1 + (-rx)*z_1
    z_2 = tz + (-ry)*x_1 + (rx)*y_1 +  (1+s)*z_1

    #Back to spherical polar coordinates from cartesian
    #Need some of the characteristics of the new ellipsoid    
    a_2, b_2 =6378137.000, 6356752.3141 #The GSR80 semi-major and semi-minor axes used for WGS84(m)
    e2_2 = 1- (b_2*b_2)/(a_2*a_2)   #The eccentricity of the GRS80 ellipsoid
    p = sqrt(x_2**2 + y_2**2)

    #Lat is obtained by an iterative proceedure:   
    lat = arctan2(z_2,(p*(1-e2_2))) #Initial value
    latold = 2*pi
    while abs(lat - latold)>10**-16: 
        lat, latold = latold, lat
        nu_2 = a_2/sqrt(1-e2_2*sin(latold)**2)
        lat = arctan2(z_2+e2_2*nu_2*sin(latold), p)

    #Lon and height are then pretty easy
    lon = arctan2(y_2,x_2)
    H = p/cos(lat) - nu_2

    #Uncomment this line if you want to print the results
    #print [(lat-lat_1)*180/pi, (lon - lon_1)*180/pi]

    #Convert to degrees
    lat = lat*180/pi
    lon = lon*180/pi

    #Job's a good'n. 
    return lat, lon

This converts a short grid ref to a six-digit one. It assumes the gridref has a two-letter prefix. It also assumes that a more useful point for conversion is the centre of the 10km or 1km square, not its south-west corner.

In [ ]:
def expand_grid_ref(gridref):
    if len(gridref) == 4:
        return gridref[0:3] + '55' + gridref[3:4] + '55'
    elif len(gridref) == 6:
        return gridref[0:4] + '5' + gridref[4:7] + '5'
    else:
        return gridref

In [ ]:
expand_grid_ref('SM90')

In [ ]:
expand_grid_ref('NN5620')

In [ ]:
to_osgb36(expand_grid_ref('SM90'))

In [ ]:
OSGB36toWGS84(*to_osgb36(expand_grid_ref('SM90')))

In [ ]:
OSGB36toWGS84(*to_osgb36(expand_grid_ref('NN5620')))

In [ ]:
assert expand_grid_ref('SM90') == 'SM955055'
assert expand_grid_ref('NN5620') == 'NN565205'
assert to_osgb36(expand_grid_ref('SM90')) == (195500, 205500)
assert OSGB36toWGS84(*to_osgb36(expand_grid_ref('SM90'))) == (51.749167440074324, -4.963575226888083)
assert OSGB36toWGS84(*to_osgb36(expand_grid_ref('NN5620'))) == (56.37769816725615, -4.325049868061924)

In [ ]:
# coords = ['SM90', 'TL65', 'SY28', 'SU62', 'TF28', 'SK83', 'NY83', 'SX37', 'SK15', 'TQ45', 'SE86', 'TF64', 'TR05', 'SP41', 'SJ57', 
#         'ST41', 'SN12', 'SU29', 'SJ07', 'SM81', 'SN02', 'SX56', 'TL13', 'SS58', 'TQ53', 'SD68', 'SM93', 'SP03', 'SN87', 'TL14', 
#         'TR36', 'SN20', 'TQ28', 'ST45', 'SX85', 'SN01', 'SJ37', 'SU81', 'TL45', 'SK25', 'TQ25', 'TF01', 'TQ05', 'ST55', 'SJ15', 
#         'NS81', 'SN71', 'SN33', 'NZ43', 'SP43', 'SE84', 'NO27', 'NT57', 'TQ57', 'SK97', 'TQ21', 'SE20', 'NN70', 'TM29', 'TQ03', 
#         'TR34', 'SP92', 'TQ29', 'SN40', 'SU67', 'SU53', 'SJ59', 'SX86', 'SN04', 'SN31', 'TL55', 'SJ26', 'NZ18', 'SD50', 'NH89', 
#         'TQ46', 'SW74', 'TQ76', 'SX87', 'TQ01', 'TQ91', 'NJ70', 'SU82', 'SN14', 'TL54', 'TL20', 'TQ15', 'TF63', 'NT46', 'SP50', 
#         'SD67', 'SU78', 'TL86', 'TQ95', 'TL03', 'SE30', 'SE50', 'SO12', 'SP32', 'SE93', 'TL77', 'ST02', 'NO30', 'SP78', 'SJ14', 
#         'NT35', 'SP33', 'SK80', 'TF27', 'SM80', 'SX99', 'SN64', 'SN34', 'SH97', 'SJ12', 'TQ85', 'SS21', 'SO51', 'NH77', 'TQ11', 
#         'SP11', 'SN72', 'SY39', 'SK67', 'TM37', 'SP93', 'TQ04', 'NT36', 'TL02', 'SH63', 'SN42', 'SX27', 'TR13', 'SE26', 'SU84', 
#         'SU75', 'SU89', 'TL32', 'SN61', 'TQ08', 'TG20', 'ST42', 'SS59', 'TL00', 'SN41', 'SN10', 'TQ42', 'ST30', 'SD47', 'TQ94', 
#         'TR15', 'ST86', 'SN78', 'TF25', 'SP91', 'TL35', 'TR24', 'TL01', 'SU76', 'TQ96', 'TQ35', 'SS19', 'TQ66', 'SK60', 'TQ67', 
#         'TQ37', 'SO66', 'SK07', 'TQ09', 'TL12', 'NS95', 'ST74', 'SN77', 'SJ06', 'SN51', 'SN79', 'TL31', 'TF00', 'TQ75', 'SU99', 
#         'TQ55', 'TQ44', 'TR25', 'ST35', 'SD71', 'SS42', 'TQ43', 'SD41', 'SO03', 'TL84', 'SW86', 'SS63', 'NO41', 'TL83', 'TL11', 
#         'SO29', 'SX58', 'SS49', 'SN95', 'SK59', 'SU59', 'SN74', 'TR03', 'SX95', 'SO11', 'SS64', 'ST11', 'SN00', 'SN03', 'TL53', 
#         'SJ08', 'SS90', 'SP77', 'SU66', 'ST32', 'TR04', 'SN68', 'NS53', 'SK90', 'SW62', 'SJ25', 'SW75', 'SW97', 'TQ31', 'SU93', 
#         'SK64', 'SS53', 'TQ32', 'SH54', 'TR06', 'SU91', 'SX76', 'TQ64', 'SJ02', 'TR26', 'ST51', 'ST60', 'SX08', 'ST31', 'SW72', 
#         'SX25', 'TQ56', 'TQ73', 'TL34', 'TL93', 'TL05', 'ST49', 'SO64', 'TQ06', 'SD30', 'SD48', 'TQ47', 'SK06', 'TQ52', 'ST64', 
#         'SP85', 'SS93', 'ST56', 'SM83', 'TR35', 'SS48', 'TG10', 'SH46', 'SH77', 'SP71', 'NR83', 'SX05', 'SK32', 'SR99', 'SX16', 
#         'SS54', 'NS61', 'SN92', 'SJ23', 'TL87', 'SJ24', 'SJ13', 'SS41', 'SD64', 'TQ54', 'SY08', 'SK53', 'SH71', 'SN54', 'TQ71', 
#         'SS68', 'SN35', 'NT07', 'SP30', 'SN30', 'NY93', 'TL21', 'SP55', 'SU94', 'NT33', 'SJ16', 'SU77', 'SM72', 'NZ71', 'SW83', 
#         'SH58', 'SH78', 'SW95', 'SE81', 'SH85', 'TQ33', 'SX84', 'SD62', 'SD61', 'NY55', 'SJ22', 'SX06', 'NS36', 'SH72', 'SH64', 
#         'SO02', 'SH61', 'SH87', 'SH34', 'SP00', 'SD45', 'SK05', 'SP75', 'SU90', 'SH53', 'ST75', 'SH96', 'ST21', 'SD65', 'SH45', 
#         'SJ17', 'SH76', 'SH55', 'SH57', 'ST62', 'NS89', 'NT53', 'SO50', 'TQ63', 'NH59', 'SP10', 'TL36', 'SN23', 'NH75', 'TQ51', 
#         'SU83', 'SP95', 'SY97', 'NZ08', 'NY78']

In [ ]:
values = [(190000, 200000),
 (560000, 250000),
 (320000, 80000),
 (460000, 120000),
 (520000, 380000),
 (480000, 330000),
 (380000, 530000),
 (230000, 70000),
 (410000, 350000),
 (540000, 150000),
 (480000, 460000),
 (560000, 340000),
 (600000, 150000),
 (440000, 210000),
 (350000, 370000),
 (340000, 110000),
 (210000, 220000),
 (420000, 190000),
 (300000, 370000),
 (180000, 210000),
 (200000, 220000),
 (250000, 60000),
 (510000, 230000),
 (250000, 180000),
 (550000, 130000),
 (360000, 480000),
 (190000, 230000),
 (400000, 230000),
 (280000, 270000),
 (510000, 240000),
 (630000, 160000),
 (220000, 200000),
 (520000, 180000),
 (340000, 150000),
 (280000, 50000),
 (200000, 210000),
 (330000, 370000),
 (480000, 110000),
 (540000, 250000),
 (420000, 350000),
 (520000, 150000),
 (500000, 310000),
 (500000, 150000),
 (350000, 150000),
 (310000, 350000),
 (280000, 610000),
 (270000, 210000),
 (230000, 230000),
 (440000, 530000),
 (440000, 230000),
 (480000, 440000),
 (320000, 770000),
 (350000, 670000),
 (550000, 170000),
 (490000, 370000),
 (520000, 110000),
 (420000, 400000),
 (270000, 700000),
 (620000, 290000),
 (500000, 130000),
 (630000, 140000),
 (490000, 220000),
 (520000, 190000),
 (240000, 200000),
 (460000, 170000),
 (450000, 130000),
 (350000, 390000),
 (280000, 60000),
 (200000, 240000),
 (230000, 210000),
 (550000, 250000),
 (320000, 360000),
 (410000, 580000),
 (350000, 400000),
 (280000, 890000),
 (540000, 160000),
 (170000, 40000),
 (570000, 160000),
 (280000, 70000),
 (500000, 110000),
 (590000, 110000),
 (370000, 800000),
 (480000, 120000),
 (210000, 240000),
 (550000, 240000),
 (520000, 200000),
 (510000, 150000),
 (560000, 330000),
 (340000, 660000),
 (450000, 200000),
 (360000, 470000),
 (470000, 180000),
 (580000, 260000),
 (590000, 150000),
 (500000, 230000),
 (430000, 400000),
 (450000, 400000),
 (310000, 220000),
 (430000, 220000),
 (490000, 430000),
 (570000, 270000),
 (300000, 120000),
 (330000, 700000),
 (470000, 280000),
 (310000, 340000),
 (330000, 650000),
 (430000, 230000),
 (480000, 300000),
 (520000, 370000),
 (180000, 200000),
 (290000, 90000),
 (260000, 240000),
 (230000, 240000),
 (290000, 370000),
 (310000, 320000),
 (580000, 150000),
 (220000, 110000),
 (350000, 210000),
 (270000, 870000),
 (510000, 110000),
 (410000, 210000),
 (270000, 220000),
 (330000, 90000),
 (460000, 370000),
 (630000, 270000),
 (490000, 230000),
 (500000, 140000),
 (330000, 660000),
 (500000, 220000),
 (260000, 330000),
 (240000, 220000),
 (220000, 70000),
 (610000, 130000),
 (420000, 460000),
 (480000, 140000),
 (470000, 150000),
 (480000, 190000),
 (530000, 220000),
 (260000, 210000),
 (500000, 180000),
 (620000, 300000),
 (340000, 120000),
 (250000, 190000),
 (500000, 200000),
 (240000, 210000),
 (210000, 200000),
 (540000, 120000),
 (330000, 100000),
 (340000, 470000),
 (590000, 140000),
 (610000, 150000),
 (380000, 160000),
 (270000, 280000),
 (520000, 350000),
 (490000, 210000),
 (530000, 250000),
 (620000, 140000),
 (500000, 210000),
 (470000, 160000),
 (590000, 160000),
 (530000, 150000),
 (210000, 190000),
 (560000, 160000),
 (460000, 300000),
 (560000, 170000),
 (530000, 170000),
 (360000, 260000),
 (400000, 370000),
 (500000, 190000),
 (510000, 220000),
 (290000, 650000),
 (370000, 140000),
 (270000, 270000),
 (300000, 360000),
 (250000, 210000),
 (270000, 290000),
 (530000, 210000),
 (500000, 300000),
 (570000, 150000),
 (490000, 190000),
 (550000, 150000),
 (540000, 140000),
 (620000, 150000),
 (330000, 150000),
 (370000, 410000),
 (240000, 120000),
 (540000, 130000),
 (340000, 410000),
 (300000, 230000),
 (580000, 240000),
 (180000, 60000),
 (260000, 130000),
 (340000, 710000),
 (580000, 230000),
 (510000, 210000),
 (320000, 290000),
 (250000, 80000),
 (240000, 190000),
 (290000, 250000),
 (450000, 390000),
 (450000, 190000),
 (270000, 240000),
 (600000, 130000),
 (290000, 50000),
 (310000, 210000),
 (260000, 140000),
 (310000, 110000),
 (200000, 200000),
 (200000, 230000),
 (550000, 230000),
 (300000, 380000),
 (290000, 100000),
 (470000, 270000),
 (460000, 160000),
 (330000, 120000),
 (600000, 140000),
 (260000, 280000),
 (250000, 630000),
 (490000, 300000),
 (160000, 20000),
 (320000, 350000),
 (170000, 50000),
 (190000, 70000),
 (530000, 110000),
 (490000, 130000),
 (460000, 340000),
 (250000, 130000),
 (530000, 120000),
 (250000, 340000),
 (600000, 160000),
 (490000, 110000),
 (270000, 60000),
 (560000, 140000),
 (300000, 320000),
 (620000, 160000),
 (350000, 110000),
 (360000, 100000),
 (200000, 80000),
 (330000, 110000),
 (170000, 20000),
 (220000, 50000),
 (550000, 160000),
 (570000, 130000),
 (530000, 240000),
 (590000, 230000),
 (500000, 250000),
 (340000, 190000),
 (360000, 240000),
 (500000, 160000),
 (330000, 400000),
 (340000, 480000),
 (540000, 170000),
 (400000, 360000),
 (550000, 120000),
 (360000, 140000),
 (480000, 250000),
 (290000, 130000),
 (350000, 160000),
 (180000, 230000),
 (630000, 150000),
 (240000, 180000),
 (610000, 300000),
 (240000, 360000),
 (270000, 370000),
 (470000, 210000),
 (180000, 630000),
 (200000, 50000),
 (430000, 320000),
 (190000, 190000),
 (210000, 60000),
 (250000, 140000),
 (260000, 610000),
 (290000, 220000),
 (320000, 330000),
 (580000, 270000),
 (320000, 340000),
 (310000, 330000),
 (240000, 110000),
 (360000, 440000),
 (550000, 140000),
 (300000, 80000),
 (450000, 330000),
 (270000, 310000),
 (250000, 240000),
 (570000, 110000),
 (260000, 180000),
 (230000, 250000),
 (300000, 670000),
 (430000, 200000),
 (230000, 200000),
 (390000, 530000),
 (520000, 210000),
 (450000, 250000),
 (490000, 140000),
 (330000, 630000),
 (310000, 360000),
 (470000, 170000),
 (170000, 220000),
 (470000, 510000),
 (180000, 30000),
 (250000, 380000),
 (270000, 380000),
 (190000, 50000),
 (480000, 410000),
 (280000, 350000),
 (530000, 130000),
 (280000, 40000),
 (360000, 420000),
 (360000, 410000),
 (350000, 550000),
 (320000, 320000),
 (200000, 60000),
 (230000, 660000),
 (270000, 320000),
 (260000, 340000),
 (300000, 220000),
 (260000, 310000),
 (280000, 370000),
 (230000, 340000),
 (400000, 200000),
 (340000, 450000),
 (400000, 350000),
 (470000, 250000),
 (490000, 100000),
 (250000, 330000),
 (370000, 150000),
 (290000, 360000),
 (320000, 110000),
 (360000, 450000),
 (240000, 350000),
 (310000, 370000),
 (270000, 360000),
 (250000, 350000),
 (250000, 370000),
 (360000, 120000),
 (280000, 690000),
 (350000, 630000),
 (350000, 200000),
 (560000, 130000),
 (250000, 890000),
 (410000, 200000),
 (530000, 260000),
 (220000, 230000),
 (270000, 850000),
 (550000, 110000),
 (480000, 130000),
 (490000, 250000),
 (390000, 70000),
 (400000, 580000),
 (370000, 580000)]

In [ ]:
#to_osgb36(coords)


In [ ]:
df =pd.read_csv('data/refs.csv')
df.head()

In [ ]:
values1 = df.test.tolist

In [ ]:
for E,N in values1:
    lat, lon = OSGB36toWGS84(float(E), float(N))
    print(str(lat)) 

In [ ]:
for E,N in values:
    lat, lon = OSGB36toWGS84(float(E), float(N))
    print(str(lon))